# Classificação das espécies de Íris

Aluno: Anderson Silva Fonseca

## Objetivo

O objetivo deste trabalho é criar um modelo de Machine Learning de classificação (ou regressão), utilizando boas práticas de programação. 
Em vez de usar os classificadores convencionais, este projeto usou uma rede neural convolucional, visando os seguintes objetivos específicos:

    * Criar um modelo que possa ser facilmente modificado sem ter que alterar outros modelos
    * Criar rotinas que sejam executadas por todos os modelos e criar rotinas específicas que só são conhecidas pelo seu próprio modelo

## User History

Como analista de espécies de plantas e árvores, quero obter um relatório completo sobre as espécies de flores encontradas na área urbana da cidade para criar um planejamento arbóreo que evite o aumento de casos de alergia nos centros da cidade e maximizem os benefícios da área verde como redução das pegadas de carbono e da temperatura.

## Detalhamento Simplificado

### Problema

Dentro do patrimônio público da cidade, temos catalogadas cada muda de flores que foi plantada em praças públicas, porém com o passar do tempo percebeu-se o aumento da quantidade de flores e cruzamento destas plantas. Com a finalidade de catalogar as novas plantas, precisamos de um modelo que seja capaz de reconhecer o tipo da planta com uma acurácia maior que 70%. Visando diminuir o tempo de análise dos especialistas sugerindo uma possível espécie de planta.

### Benefícios

Reduzir o tempo de campo dos especialistas em condições desfavoráveis (sol e transporte)
Estudar as consequências do aumento de plantas do gênero na cidade

### Trade-off

É melhor remover possíveis plantas que estão classificadas incorretamente do que deixar uma planta que possa degradar a saúde da população.

### Dados

Os dados podem ser obtidos a partir das imagens privadas retiradas das praças públicas durante o sensoriamento arbóreo. Os dados não precisam ser atualizados com uma frequência muito alto, mas recomenda-se atualizá-la a cada 6 meses.

### Obtenção dos resultados

O resultado da classificação do modelo pode ser obtido durante a categorização das plantas do centro urbano.

### Entrada e saída dos dados

A entrada dos dados são imagens das flores tiradas durante o dia. A saída será um valor único indicado qual a espécie determinada flor pertence.

### Algoritmos

Gostaríamos de testar os algoritmos de classificação de imagem mais conhecidas atualmente com diferentes hiperparâmetros

### Baseline

Como modelo base, foi sugerido usar modelos de classificação que são usados com o dataset Iris

### Gold Dataset

Um bom dataset para nosso modelo é que os dados estejam corretamente catalogados com suas determinadas espécies

In [ ]:
# Imports
from matplotlib import pyplot as plt
import numpy as np

from src.datasets.iris_dataset import IrisDataset
from src.preprocessing.image_pre_processor import ImagePreProcessor
from src.models.unet import UNet
from src.evaluation.evaluator import Evaluator
from src.runner.runner import Runner

## Coleta e Análise dos dados

### Obtenção dos dados

Os dados foram obtidos a partir de um dataset disponibilizado no [kaggle](https://www.kaggle.com/datasets/jeffheaton/iris-computer-vision?resource=download).

> Por se tratar de um dataset de imagens não é possível usar a API para descompactar as imagens, então seus dados foram baixados e colocados dentro da pasta `data/iris/`

### Código-Fonte

#### Classe DatasetLoader

A classe `DatasetLoader` é uma classe abstrata que engloba as principais funções que toda classe que carrega dataset deve ter. No contexto atual, ela responsável pode recuperar o dado bruto, o conjunto de labels e os respectivos conjuntos de dados para `x` e `y`, ou features e label respectivamente.

#### Classe DatasetLoader

A classe `IrisDataset` é uma classe que herda de `DatasetLoader`. Foi feita uma sobrecarga nos métodos para que ela seja capaz de acessar o dataset da Iris e que ela conheça todas as labels relacionadas.

### Exploração dos dados


In [ ]:


iris_dataset = IrisDataset("data/iris")
example_images = iris_dataset.get_x()[:10]

fig = plt.figure(figsize =(12,5))
columns = 5
rows = 2

for i in range(len(example_images)):
    ax = fig.add_subplot(rows, columns, i)
    plt.imshow(example_images[i])

plt.show()

In [ ]:
print(f"Labels: {iris_dataset.get_label_names()}")

In [ ]:
print("Distribuição dos dados")

labels, counts = np.unique(iris_dataset.get_y(), return_counts = True)

plt.figure(figsize=(10, 5))
fig, ax = plt.subplots()
bars = ax.barh(iris_dataset.get_label_names(), counts)

ax.bar_label(bars)

## Pré-processamento

### Código-Fonte

#### Classe PreProcessor

A classe `PreProcessor` é uma classe abstrata que engloba as principais para realizar o pré-processamento No contexto atual, ela responsável por aplicar as transformações básicas em `x` e `y`. Atualmente ela também é responsável por fazer a divisão de treino/teste/validação.

#### Classe ImagePreProcessor

A classe `ImagePreProcessor` é uma classe que herda de `PreProcessor` e tem como principal característica fazer o tratamento de dados baseados em imagem. Sua principal caracteristica é que ela é capaz de ler as imagens, fazer uma normalização baseada nos canais e realizar uma reamostragem em cada imagem do dataset.

### Resultados do pré-processamento

In [ ]:
preprocessor = ImagePreProcessor(iris_dataset)

In [ ]:
print(f"Original Shape: {example_images[0].shape}")
print(f"Processed Shape: {preprocessor.get_x()[0].shape}")
print(f"Original Min: {np.min(iris_dataset.get_x())} - Original Max: {np.max(iris_dataset.get_x())}")
print(f"Original Min: {np.min(preprocessor.get_x())} - Original Max: {np.max(preprocessor.get_x())}")

## Modelos de Classificação

Os modelos de classificação utilizados nesse trabalho são baseados usando o framework `tensorflow`.

### Código Fonte

#### Classe BaseModel

A classe `BaseModel` é uma classe abstrata que visa encapsular os componentes essenciais de um modelo de rede neural baseado no tensorflow. Além disso, esta classe foi adaptada para ter as seguintes funcionalidades adicionais.

* Identificação única de treinamentos

    Toda classe que herda de `BaseModel` recebe um identificador único durante o treinamento, facilitando dos logs gerados pela aplicação

* Painel Tensorboard
    
    Durante o treinamento de um modelo baseado no `BaseModel`, por padrão é criado uma callback que armazena os logs do treinamento dentro do tensorboard. As métricas são armazenadas em uma pasta dentro dos logs com identificado único do treinamento.

    > Caso o usuário já tenha criado sua instância do tensorboard, esta funcionalidade é desativada para evitar demoras na execução

* Model Checkpoint
    
    Por padrão, qualquer treinamento realizado no BaseModel possui os seus pesos salvos automaticamente usando um ModelCheckpoint. Seus pesos são salvos dentro do log.

* Sugestão de hiperparâmetros
    
    O modelo pode inferir os hiperparâmetros que podem ser alterados

> Todas essas funcionalidades criadas podem ser desativadas com argumentos fornecidos na documentação

#### Classe UNet

A classe UNet herda as funcionalidades de `BaseModel`. Ela apresenta dentro de sim todas as camadas para a criação da UNet baseada no artigo. 

In [ ]:
model = UNet()

## Métodos de Avaliação

Para avaliar o modelo foram usadas as métricas acurácia, precisão, recall, f1 score e a matriz de confusão.

### Código Fonte

#### Classe Evaluator

A classe `Evaluator` possui um conjunto de métodos que calculam as métricas a partir do resultados preditos e os dados verdadeiros. Ela possui também funções para salvar a matriz de confusão e as métricas em um arquivo.


In [1]:

evaluator = Evaluator()


## Execução do modelo

Para executar o modelo foi criada uma classe `Runner` que recebe o dataset original, o preprocessador, o modelo-alvo, e um conjunto de métricas de avaliação

### Código Fonte

#### Classe Runner

A classe `Runner` possui todo um conjunto de métodos para executar o pipeline do tensorflow para a geração do modelo. Durante sua inicialização, os dados são preparados para execução. Sua função `run()` engloba as seguintes etapas

1. Recuperar hiperparâmetros do modelo-alvo. (Ex.: nº de filtros, batch_normalization, etc)
2. Recuperar hiperparâmetros de compilação do modelo (Ex.: Otimizadores, nº de épocas,  etc)
3. Geração de hiperparâmetros e suas variações. Cria todas as possibilidades de hiperparâmetros
4. Gera o modelo com os hiperparâmetro escolhidos.
5. Realiza o treinamento
6. Recupera os identificadores únicos de treinamento de cada modelo
7. Realiza a predição
8. Usa o conjunto de métricas de avaliação fornecido para gerar os resultados
9. Salva as métricas e parâmetros dentro da pasta do modelos gerados.
10. Retorna um dataframe para análise dos resultados

In [2]:
runner = Runner(iris_dataset, preprocessor, model, evaluator)

In [3]:
results = runner.run()

Epoch 1/150
19/19 [==============================] - 15s 506ms/step - loss: 0.9277 - accuracy: 0.6258 - val_loss: 1.0250 - val_accuracy: 0.7353
Epoch 2/150
19/19 [==============================] - 6s 342ms/step - loss: 0.8699 - accuracy: 0.6490 - val_loss: 1.0275 - val_accuracy: 0.7647
Epoch 3/150
19/19 [==============================] - 4s 207ms/step - loss: 0.8504 - accuracy: 0.6424 - val_loss: 0.9863 - val_accuracy: 0.7647
Epoch 4/150
19/19 [==============================] - 4s 212ms/step - loss: 0.7966 - accuracy: 0.6556 - val_loss: 0.9340 - val_accuracy: 0.7647
Epoch 5/150
19/19 [==============================] - 4s 221ms/step - loss: 0.7687 - accuracy: 0.6689 - val_loss: 0.8753 - val_accuracy: 0.7353
Epoch 6/150
19/19 [==============================] - 4s 203ms/step - loss: 0.7531 - accuracy: 0.6656 - val_loss: 0.9117 - val_accuracy: 0.7647
Epoch 7/150
19/19 [==============================] - 4s 218ms/step - loss: 0.7393 - accuracy: 0.6689 - val_loss: 0.8568 - val_accuracy: 0.705

d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/150
19/19 [==============================] - 7s 277ms/step - loss: 1.0465 - accuracy: 0.3576 - val_loss: 1.0075 - val_accuracy: 0.7647
Epoch 2/150
19/19 [==============================] - 4s 210ms/step - loss: 0.9355 - accuracy: 0.6490 - val_loss: 0.9640 - val_accuracy: 0.7353
Epoch 3/150
19/19 [==============================] - 4s 204ms/step - loss: 0.8956 - accuracy: 0.6490 - val_loss: 0.9414 - val_accuracy: 0.7353
Epoch 4/150
19/19 [==============================] - 4s 213ms/step - loss: 0.8804 - accuracy: 0.6490 - val_loss: 0.9282 - val_accuracy: 0.7353
Epoch 5/150
19/19 [==============================] - 4s 207ms/step - loss: 0.8733 - accuracy: 0.6490 - val_loss: 0.9185 - val_accuracy: 0.7353
Epoch 6/150
19/19 [==============================] - 4s 209ms/step - loss: 0.8690 - accuracy: 0.6490 - val_loss: 0.9094 - val_accuracy: 0.7353
Epoch 7/150
19/19 [==============================] - 4s 206ms/step - loss: 0.8660 - accuracy: 0.6490 - val_loss: 0.8998 - val_accuracy: 0.7353

d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/150
19/19 [==============================] - 10s 433ms/step - loss: 0.9335 - accuracy: 0.6490 - val_loss: 0.9661 - val_accuracy: 0.7353
Epoch 2/150
19/19 [==============================] - 8s 428ms/step - loss: 0.9288 - accuracy: 0.6490 - val_loss: 0.9666 - val_accuracy: 0.7353
Epoch 3/150
19/19 [==============================] - 7s 395ms/step - loss: 0.9247 - accuracy: 0.6490 - val_loss: 0.9628 - val_accuracy: 0.7353
Epoch 4/150
19/19 [==============================] - 7s 397ms/step - loss: 0.9210 - accuracy: 0.6490 - val_loss: 0.9558 - val_accuracy: 0.7353
Epoch 5/150
19/19 [==============================] - 7s 401ms/step - loss: 0.9177 - accuracy: 0.6490 - val_loss: 0.9467 - val_accuracy: 0.7353
Epoch 6/150
19/19 [==============================] - 8s 402ms/step - loss: 0.9149 - accuracy: 0.6490 - val_loss: 0.9360 - val_accuracy: 0.7353
Epoch 7/150
19/19 [==============================] - 7s 402ms/step - loss: 0.9122 - accuracy: 0.6490 - val_loss: 0.9244 - val_accuracy: 0.735

d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/150
19/19 [==============================] - 25s 1s/step - loss: 0.9548 - accuracy: 0.6258 - val_loss: 0.7838 - val_accuracy: 0.7353
Epoch 2/150
19/19 [==============================] - 15s 779ms/step - loss: 0.8703 - accuracy: 0.6325 - val_loss: 0.9717 - val_accuracy: 0.7353
Epoch 3/150
19/19 [==============================] - 14s 733ms/step - loss: 0.8565 - accuracy: 0.6424 - val_loss: 0.9366 - val_accuracy: 0.7353
Epoch 4/150
19/19 [==============================] - 14s 769ms/step - loss: 0.8591 - accuracy: 0.6457 - val_loss: 0.8839 - val_accuracy: 0.7353
Epoch 5/150
19/19 [==============================] - 14s 769ms/step - loss: 0.8467 - accuracy: 0.6589 - val_loss: 0.8910 - val_accuracy: 0.7353
Epoch 6/150
19/19 [==============================] - 14s 774ms/step - loss: 1.0291 - accuracy: 0.6424 - val_loss: 0.8696 - val_accuracy: 0.7353
Epoch 7/150
19/19 [==============================] - 14s 776ms/step - loss: 0.8306 - accuracy: 0.6490 - val_loss: 1.3234 - val_accuracy: 0.

d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Documents\Workspaces\Python\image-classification\.venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
results

,n_filters,optimizer,learning_rate,epochs,name,accuracy,precision,recall,f1_score
0,32,Adam,0.0010,150,U-Net,0.552941,0.537152,0.552941,0.532571
1,32,Adam,0.0001,150,U-Net,0.576471,0.522689,0.576471,0.432917
2,32,SGD,0.0010,150,U-Net,0.600000,0.530560,0.600000,0.477014
3,32,SGD,0.0001,150,U-Net,0.564706,0.318893,0.564706,0.407607
4,64,Adam,0.0010,150,U-Net,0.600000,0.657919,0.600000,0.497809
5,64,Adam,0.0001,150,U-Net,0.482353,0.452393,0.482353,0.461941
6,64,SGD,0.0010,150,U-Net,0.611765,0.614000,0.611765,0.533823
7,64,SGD,0.0001,150,U-Net,0.564706,0.318893,0.564706,0.407607


## Implantação do Modelo e Geração de Valor

A partir dos dados obtidos é possível visualizar qual a melhor combinação de hiperparâmetros para o modelo de rede neural usado. Por exemplo, podemos usar o modelo que teve maior acurácia ou um que se ajustou aos objetivos do trabalho (acurácia maior que 70%)

### Implantação
Após escolher o modelo, sua implantação pode ser feita a partir de uma API web onde o usuário enviar uma fotografia de planta e recebe como resultado a espécie planta sugerida pelo modelo. Essa API pode ser facilmente integrada a outros sistemas, por exemplo o sistema de cadastramento de mudas de plantas, ou sistemas que precisam catalogar as flores adquiridas de maneira rápida e eficiente.

### Calculo de Ganhos
Os ganhos positivos podem ser calculadas a medida que os usuários usam o modelo para facilitar
* O tempo de identificação das amostras de plantas
* Os gastos com alimentação e transporte de um funcionário especializado para o local

### Monitoramento
O monitoramento pode ser feito de maneira constante junto com os dados que estão sendo enviados pela API do sistema. Caso a acurácia atual esteja menor que o padrão desejado pode ser necessário refazer o treinamento com dados mais atualizados.


